# ONNX model export

In this notebook we'll cover how to export a setfit model to ONNX format.  This is useful for deploying models in production and speeding up inference on CPUs.

## Setup

If you're running this Notebook on Colab or some other cloud platform, you will need to install a few libraries namely `setfit`, `onnxruntime`, `onnx`, and `skl2onnx`. Uncomment the following cell and run it:

In [ ]:
# %pip install setfit onnxruntime onnx skl2onnx

## Convert model to ONNX

There are two types of heads available for setfit models. Sklearn based estimators and `torch.nn.Module`. We'll convert the example from `text-classification.ipynb` which uses an `sklearn` head and a base model that hasn't been trained which defaults to a `nn.Module` head.  We start by loading in the models

In [ ]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("lewtun/my-awesome-setfit-model")

Next we import the onnx_export onnx function from `setfit.onnx` and run the export.  The export function takes in the `model_body` and the `model_head` from the setfit model loaded above.

In [ ]:
from setfit.onnx import export_onnx

# Export the sklearn based model
output_path = "sklearn_model.onnx"
export_onnx(model.model_body,
            model.model_head,
            opset=12,
            output_path=output_path)

Excellent! Let's check to see that this worked. If we list our current directory we should see a `model.onnx` file if everything succeeded. If you're interested in looking at the internals of this file check out [netron](https://github.com/lutzroeder/netron). 

In [ ]:
import os
assert output_path in os.listdir() 

With our model successfully converted to `onnx` let's run some initial predictions and make sure we are getting the same results as our earlier model.  We'll start by predicting using the original pytorch pipeline.

In [ ]:
# Run inference using the original model
input_text = ["i loved the spiderman movie!", "pineapple on pizza is the worst 🤮"]
pytorch_preds = model(input_text)
pytorch_preds 

Now let's make some predictions using the onnx model that we just created!

In [ ]:
import onnxruntime
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("lewtun/my-awesome-setfit-model")
inputs = tokenizer(
    input_text,
    padding=True,
    truncation=True,
    return_attention_mask=True,
    return_token_type_ids=True,
    return_tensors="np",
)

session = onnxruntime.InferenceSession(output_path)

onnx_preds = session.run(None, dict(inputs))[0]

In [ ]:
onnx_preds

The predicted outputs should look the same let's verify using numpy's `array_equal` function.

In [ ]:
import numpy as np
assert np.array_equal(onnx_preds, pytorch_preds)

Nice! The output from our original pytorch model and our new onnx optimized model is the same!